## Monoid and Semigroup

### Monoid

In abstract algebra, a monoid is a mathematical structure with two key properties:

1. **Associative binary operation:** It involves a set of elements and a binary operation that combines any two elements from the set. This operation must be associative, meaning $(a * b) * c  = a * (b * c)$ for all elements $a$, $b$, and $c$ in the set. Here, "*" represents the binary operation.
2. **Identity element:** The set must also contain a special element called the identity element, denoted by $e$ (or sometimes $1$). This element has the property that for any element $a$ in the set, $a * e = e * a = a$.

Here are some common examples of monoids:

* **Non-negative integers $(0, 1, 2, ...)$ with addition:**  In this case, addition $(+)$ is the binary operation, and $0$ acts as the identity element. $(0 + a = a + 0 = a)$ for any non-negative integer $a$.
* **Positive integers $(1, 2, 3, ...)$ with multiplication:** Here, multiplication ($\times$) is the binary operation, and $1$ acts as the identity element $(1 \times a = a \times 1 = a)$ for any positive integer $a$. Note that this monoid is different from the non-negative integers with multiplication because $0$ is not included, which breaks the associative property $(0 \times a ≠ a \times 0)$ for some positive integers $a$.
* **Boolean set with $\land$:**  The Boolean set $\{T, F\}$ is a monoid with the binary operation $\land$ and the identity element $T$.

It's important to note that unlike groups (a more complex structure), monoids don't necessarily have inverses for all elements. In other words, you might not always be able to "undo" an operation using another element.

### Monoid Typeclass

In Haskell, the `Monoid` typeclass captures the essence of mathematical monoids for working with data types. It provides a way to represent types that can be combined using an associative operation and have an identity element.

Here's how the `Monoid` typeclass works in Haskell:

* **Methods:**
    * `mempty :: m` - This function defines the identity element for the monoid. It's the element that doesn't affect the result when combined with others using the binary operation.
    * `mappend :: m -> m -> m` (often written as `<>`) - This function represents the binary operation for combining two elements of the monoid type `m`.
    * `mconcat :: [m] -> m` (optional) - This function folds a list of elements of type `m` into a single value using `mappend` and `mempty`. By default, it's defined as `foldr mappend mempty`.

**Examples:**

1. **List Monoid:**

Consider lists (`[a]`) as a monoid.
* `mempty` is the empty list `[]`.
* `mappend` (written as `<>`) is list concatenation (`++`).
   
   ```haskell
   instance Monoid [] where
       mempty = []
       xs <> ys = xs ++ ys
   ```

These are just a few examples. The `Data.Monoid` module in Haskell provides monoid instances for many common data types like strings and sums.

By defining the `Monoid` typeclass for a data type, you gain access to powerful functions like `foldl` and `foldr` that can efficiently combine elements of that type using the defined operation and identity element. This makes working with sequences and collections more convenient and expressive in Haskell.

### Exercise: Optional Monoid

Write the Monoid instance for our `Maybe` type renamed to `Optional`.

```haskell
data Optional a =
    Nada
  | Only a
  deriving (Eq, Show)
```

In [ ]:
data Optional a =
    Nada
  | Only a
  deriving (Eq, Show)

instance (Semigroup a) => Semigroup (Optional a) where
    (Only x) <> (Only y) = Only (x <> y)
    (Only x) <> Nada = Only x
    Nada <> (Only x) = Only x
    Nada <> Nada = Nada

instance (Monoid a) => Monoid (Optional a) where
    mempty = Nada

In [ ]:
Only "Hello " <> Only "World"

Only "Hello World"

In [ ]:
Nada <> Nada

Nada

### Using `QuickCheck` to verify monid laws

We can use `QuickCheck` to verify the monoid laws.

```haskell
import Data.Monoid
import Test.QuickCheck

monoidAssoc :: (Eq m, Monoid m) => m -> m -> m -> Bool
monoidAssoc a b c =
    (a <> (b <> c)) == ((a <> b) <> c)



monoidLeftIdentity :: (Eq m, Monoid m) => m -> Bool
monoidLeftIdentity a = (mempty <> a) == a

monoidRightIdentity :: (Eq m, Monoid m) => m -> Bool
monoidRightIdentity a = (a <> mempty) == a
```

### Exercise: Maybe Another Monoid

Write a `Monoid` instance for a `Maybe` type which doesn’t require a `Monoid`
for the contents. Reuse the `Monoid` law `QuickCheck` properties and use them to validate the instance.
Don’t forget to write an `Arbitrary` instance for `First'`. We won’t
always stub that out explicitly for you. We suggest learning how to use the `frequency` function from `QuickCheck` for `First'`’s instance.

> <span style="color:green">**Answer:**</span>

```haskell
module MaybeAnotherMonoid where

import Test.QuickCheck

data Optional a
  = Nada
  | Only a
  deriving (Eq, Show)

newtype First' a = First' {getFirst' :: Optional a}
  deriving (Eq, Show)

instance Semigroup (First' a) where
  (<>) :: First' a -> First' a -> First' a
  First' (Only x) <> _ = First' (Only x)
  First' Nada <> First' (Only x) = First' (Only x)
  _ <> _ = First' Nada

instance Monoid (First' a) where
  mempty :: First' a
  mempty = First' Nada

instance (Arbitrary a) => Arbitrary (First' a) where
  arbitrary = do
    x <- arbitrary
    frequency
      [ (1, return First' {getFirst' = Nada}),
        (3, return $ First' {getFirst' = Only x})
      ]

type FirstMappend =
  First' String ->
  First' String ->
  First' String ->
  Bool

type FirstId = First' String -> Bool

monoidAssoc :: (Eq m, Monoid m) => m -> m -> m -> Bool
monoidAssoc a b c =
  (a <> (b <> c)) == ((a <> b) <> c)

monoidLeftIdentity :: (Eq m, Monoid m) => m -> Bool
monoidLeftIdentity a = (mempty <> a) == a

monoidRightIdentity :: (Eq m, Monoid m) => m -> Bool
monoidRightIdentity a = (a <> mempty) == a

main :: IO ()
main = do
  quickCheck (monoidAssoc :: FirstMappend)
  quickCheck (monoidLeftIdentity :: FirstId)
  quickCheck (monoidRightIdentity :: FirstId)
```

### Semigroup

**What is a Semigroup?**

In Haskell, the `Semigroup` typeclass represents types that can be combined using an associative binary operation. This means:

- There's a binary operator (`<>`) that takes two values of the same type and combines them.
- The operation is associative: `(a <> b) <> c` is equivalent to `a <> (b <> c)`. The order of parentheses doesn't affect the result as long as you combine elements from left to right.

**Common Examples of Semigroups:**

- **Numbers:** Addition (`(+)`) and multiplication (`(*)`) are both associative operations, making `Int`, `Float`, and other numeric types instances of `Semigroup`.
- **Lists:** Appending (`++`) lists is associative, making `[a]` a `Semigroup` instance.
- **Strings:** Concatenation (`++`) is associative, so `String` is a `Semigroup` instance.
- **Booleans:** Taking the logical OR (`||`) is associative, so `Bool` is a `Semigroup` instance (combines to `True` if any element is `True`).

**Implementing Semigroup for Custom Types:**

You can define a `Semigroup` instance for your own data types by providing an implementation for the `<>` operator that adheres to associativity. Here's an example:

```haskell
data Max a = Max a a deriving (Show)

instance Semigroup (Max a) where
  Max x y <> Max z w = Max (max x z) (max y w)
```

This `Max` type holds two values and the `<>` operator combines two `Max` instances by taking the maximum of their respective elements.


**Key Points:**

- `Semigroup` provides a generic way to combine elements of a type.
- Associativity is crucial for using `<>` effectively.
- You can define custom `Semigroup` instances for your data types.
- Functions can leverage `Semigroup` to work on various types that can be combined.

By understanding the `Semigroup` typeclass, you can write more generic and flexible Haskell code that works with different data types that share the concept of combining elements.